In [44]:
import pandas as pd
import numpy as np
euadr_dataset = pd.read_csv('./EUADR_target_disease.csv',encoding = "ISO-8859-1",sep="\t")

In [45]:
euadr_dataset

,ASSOCIATION_TYPE,PMID,NUM_SENTENCE,ENTITY1_TEXT,ENTITY1_INI,ENTITY1_END,ENTITY1_TYPE,ENTITY2_TEXT,ENTITY2_INI,ENTITY2_END,ENTITY2_TYPE,SENTENCE
0,SA,17241106,16,LRP5,19,23,Genes & Molecular Sequences,osteoporosis,80,92,Diseases & Disorders,Our work supported LRP5 genetic variants as po...
1,SA,17241106,17,rs491347,20,28,SNP & Sequence variations,osteoporosis,108,120,Diseases & Disorders,"Especially, the SNP rs491347 and its strongly ..."
2,SA,17241106,17,rs1784235,69,78,SNP & Sequence variations,osteoporosis,108,120,Diseases & Disorders,"Especially, the SNP rs491347 and its strongly ..."
3,FA,18697826,0,fetal haemoglobin,93,110,Genes & Molecular Sequences,beta-thalassaemia,132,149,Diseases & Disorders,The HBS1L-MYB intergenic region on chromosome ...
4,PA,18697826,1,HbF,19,22,Genes & Molecular Sequences,HBB disorders,64,77,Diseases & Disorders,Fetal haemoglobin (HbF) level modifies the cli...
...,...,...,...,...,...,...,...,...,...,...,...,...
350,PA,19110009,13,fetuin-A,5,13,Genes & Molecular Sequences,aortic stiffness,60,76,Diseases & Disorders,Male fetuin-A SerSer carriers demonstrate part...
351,PA,19110009,4,fetuin-A,4,12,Genes & Molecular Sequences,vascular calcification,95,117,Diseases & Disorders,The fetuin-A polymorphism (Thr256Ser) is assoc...
352,PA,19110611,2,EGFR,0,4,Genes & Molecular Sequences,pancreatic adenocarcinoma,24,49,Diseases & Disorders,EGFR gene expression in pancreatic adenocarcin...
353,PA,19110611,0,epidermal growth factor,8,31,Genes & Molecular Sequences,pancreatic carcinoma,139,159,Diseases & Disorders,Role of epidermal growth factor gene in the de...


In [46]:
euadr_dataset.ASSOCIATION_TYPE.value_counts()

ASSOCIATION_TYPE
PA    213
FA     93
SA     30
Name: count, dtype: int64

In [47]:
df =euadr_dataset.copy()

In [75]:
import pandas as pd
df_train = pd.read_csv("/home/016651544/gemma/datasets/gad/test_data.tsv",sep= "\t")
df_train.head()

,sentence,label
0,These results suggest that the C1772T polymorp...,True
1,"In our setting, @DISEASE$ among alcoholic indi...",True
2,MPO genotype GG is associated with @DISEASE$ i...,True
3,These three studies do not provide consistent ...,True
4,Our prospective findings suggest that individu...,True


In [76]:
df_train.label.value_counts()

label
True     2801
False    2529
Name: count, dtype: int64

In [77]:
df_train.shape

(5330, 2)

In [78]:
df_2 = pd.read_csv("/home/016651544/llama2_euadr/test_df_10_25_FT_model_euadr.csv")

In [51]:
df_2.columns

Index(['sentence', 'actual_output', 'predicted_output'], dtype='object')

In [52]:
df_2.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,True


In [53]:
# Function to mask entities in a sentence
def mask_entities(sentence, ent1_text, ent1_ini, ent1_end, ent1_type, ent2_text, ent2_ini, ent2_end, ent2_type):
    # Sort entities by their position to avoid offset issues when replacing
    entities = sorted(
        [(ent1_text, ent1_ini, ent1_end, ent1_type), 
         (ent2_text, ent2_ini, ent2_end, ent2_type)], 
        key=lambda x: x[1], reverse=True
    )
    
    # Replace entities starting from the end of the sentence
    for text, start, end, etype in entities:
        mask = '@GENE$' if 'Gene' in etype else '@DISEASE$'
        sentence = sentence[:start] + mask + sentence[end:]
        
    return sentence

# Apply the masking function to each row
df['SENTENCE_MASKED'] = df.apply(lambda x: mask_entities(
    x['SENTENCE'], x['ENTITY1_TEXT'], x['ENTITY1_INI'], x['ENTITY1_END'], x['ENTITY1_TYPE'],
    x['ENTITY2_TEXT'], x['ENTITY2_INI'], x['ENTITY2_END'], x['ENTITY2_TYPE']
), axis=1)

df[['SENTENCE', 'SENTENCE_MASKED',"ASSOCIATION_TYPE"]]

,SENTENCE,SENTENCE_MASKED,ASSOCIATION_TYPE
0,Our work supported LRP5 genetic variants as po...,Our work supported @GENE$ genetic variants as ...,SA
1,"Especially, the SNP rs491347 and its strongly ...","Especially, the SNP @DISEASE$ and its strongly...",SA
2,"Especially, the SNP rs491347 and its strongly ...","Especially, the SNP rs491347 and its strongly ...",SA
3,The HBS1L-MYB intergenic region on chromosome ...,The HBS1L-MYB intergenic region on chromosome ...,FA
4,Fetal haemoglobin (HbF) level modifies the cli...,Fetal haemoglobin (@GENE$) level modifies the ...,PA
...,...,...,...
350,Male fetuin-A SerSer carriers demonstrate part...,Male @GENE$ SerSer carriers demonstrate partic...,PA
351,The fetuin-A polymorphism (Thr256Ser) is assoc...,The @GENE$ polymorphism (Thr256Ser) is associa...,PA
352,EGFR gene expression in pancreatic adenocarcin...,@GENE$ gene expression in @DISEASE$ cells sign...,PA
353,Role of epidermal growth factor gene in the de...,Role of @GENE$ gene in the development of panc...,PA


In [61]:
euadr_unmasked = pd.concat([df[['SENTENCE', 'SENTENCE_MASKED',"ASSOCIATION_TYPE","ENTITY1_TEXT","ENTITY2_TEXT"]], df_2[["actual_output","predicted_output"]]], axis = 1)#.rename(columns = {"sentence":"SENTENCE"})],axis = 0 )
# pd.merge(df[['SENTENCE', 'SENTENCE_MASKED',"ASSOCIATION_TYPE"]], df_2.rename(columns = {"sentence":"SENTENCE"}), on =['SENTENCE'], how='left' )

In [62]:
euadr_unmasked
# euadr_unmasked.to_csv("./euadr_unmasked.csv",index=False)

,SENTENCE,SENTENCE_MASKED,ASSOCIATION_TYPE,ENTITY1_TEXT,ENTITY2_TEXT,actual_output,predicted_output
0,Our work supported LRP5 genetic variants as po...,Our work supported @GENE$ genetic variants as ...,SA,LRP5,osteoporosis,True,True
1,"Especially, the SNP rs491347 and its strongly ...","Especially, the SNP @DISEASE$ and its strongly...",SA,rs491347,osteoporosis,True,True
2,"Especially, the SNP rs491347 and its strongly ...","Especially, the SNP rs491347 and its strongly ...",SA,rs1784235,osteoporosis,True,True
3,The HBS1L-MYB intergenic region on chromosome ...,The HBS1L-MYB intergenic region on chromosome ...,FA,fetal haemoglobin,beta-thalassaemia,False,False
4,Fetal haemoglobin (HbF) level modifies the cli...,Fetal haemoglobin (@GENE$) level modifies the ...,PA,HbF,HBB disorders,True,True
...,...,...,...,...,...,...,...
350,Male fetuin-A SerSer carriers demonstrate part...,Male @GENE$ SerSer carriers demonstrate partic...,PA,fetuin-A,aortic stiffness,True,False
351,The fetuin-A polymorphism (Thr256Ser) is assoc...,The @GENE$ polymorphism (Thr256Ser) is associa...,PA,fetuin-A,vascular calcification,True,True
352,EGFR gene expression in pancreatic adenocarcin...,@GENE$ gene expression in @DISEASE$ cells sign...,PA,EGFR,pancreatic adenocarcinoma,True,False
353,Role of epidermal growth factor gene in the de...,Role of @GENE$ gene in the development of panc...,PA,epidermal growth factor,pancreatic carcinoma,True,True


In [65]:
euadr_unmasked.actual_output.value_counts()

actual_output
True     262
False     93
Name: count, dtype: int64

In [93]:
seed = 144

In [94]:
# Function to sample the data
def sample_data(df, percentage):
    n_samples = int(len(df) * percentage)
    n_true = int(n_samples * 0.7)
    n_false = n_samples - n_true

    true_samples = df[df['predicted_output'] == True].sample(n=min(n_true, len(df[df['predicted_output'] == True])), random_state=seed)
    false_samples = df[df['predicted_output'] == False].sample(n=min(n_false, len(df[df['predicted_output'] == False])), random_state=seed)

    # If there aren't enough True or False samples, fill with the other type
    if len(true_samples) < n_true:
        additional_samples = df[df['predicted_output'] == False].sample(n=(n_true - len(true_samples)), random_state=seed)
        true_samples = pd.concat([true_samples, additional_samples])
    elif len(false_samples) < n_false:
        additional_samples = df[df['predicted_output'] == True].sample(n=(n_false - len(false_samples)), random_state=seed)
        false_samples = pd.concat([false_samples, additional_samples])
    print("# of True and False samples :",len(true_samples), len(false_samples), round(len(true_samples)/n_samples,2)*100, round(len(false_samples)/n_samples,2)*100 )
    return pd.concat([true_samples, false_samples]).sample(frac=1, random_state=seed)  # Shuffle the dataset

df = euadr_unmasked.copy()
# Sampling 20%, 50%, 80%, and 100% of the data
sample_10 = sample_data(df, 0.1)
sample_20 = sample_data(df, 0.2)
sample_30 = sample_data(df, 0.3)
sample_40 = sample_data(df, 0.4)
sample_50 = sample_data(df, 0.5)
sample_60 = sample_data(df, 0.6)
sample_70 = sample_data(df, 0.7)
sample_80 = sample_data(df, 0.8)
sample_90 = sample_data(df, 0.9)
sample_100 = sample_data(df, 1.0)

# Example: print the sampled data sizes
print("Sampled 10%:", len(sample_10))
print("Sampled 20%:", len(sample_20))
print("Sampled 30%:", len(sample_30))
print("Sampled 40%:", len(sample_40))
print("Sampled 50%:", len(sample_50))
print("Sampled 60%:", len(sample_60))
print("Sampled 70%:", len(sample_70))
print("Sampled 80%:", len(sample_80))
print("Sampled 90%:", len(sample_90))
print("Sampled 100%:", len(sample_100))

# of True and False samples : 24 11 69.0 31.0
# of True and False samples : 49 22 69.0 31.0
# of True and False samples : 74 32 70.0 30.0
# of True and False samples : 99 43 70.0 30.0
# of True and False samples : 123 54 69.0 31.0
# of True and False samples : 149 64 70.0 30.0
# of True and False samples : 173 75 70.0 30.0
# of True and False samples : 198 86 70.0 30.0
# of True and False samples : 223 96 70.0 30.0
# of True and False samples : 248 107 70.0 30.0
Sampled 10%: 35
Sampled 20%: 71
Sampled 30%: 106
Sampled 40%: 142
Sampled 50%: 177
Sampled 60%: 213
Sampled 70%: 248
Sampled 80%: 284
Sampled 90%: 319
Sampled 100%: 355


In [95]:
!pwd

/home/016651544/gemma


In [96]:
"/home/016651544/llama2_euadr/test_df_10_25_FT_model_euadr.csv"

'/home/016651544/llama2_euadr/test_df_10_25_FT_model_euadr.csv'

In [97]:
sample_10.to_csv(f"./datasets/{seed}/euadr/10_sample_val_euadr.csv",index=True)
sample_20.to_csv(f"./datasets/{seed}/euadr/20_sample_val_euadr.csv",index=True)
sample_30.to_csv(f"./datasets/{seed}/euadr/30_sample_val_euadr.csv",index=True)
sample_40.to_csv(f"./datasets/{seed}/euadr/40_sample_val_euadr.csv",index=True)
sample_50.to_csv(f"./datasets/{seed}/euadr/50_sample_val_euadr.csv",index=True)
sample_60.to_csv(f"./datasets/{seed}/euadr/60_sample_val_euadr.csv",index=True)
sample_70.to_csv(f"./datasets/{seed}/euadr/70_sample_val_euadr.csv",index=True)
sample_80.to_csv(f"./datasets/{seed}/euadr/80_sample_val_euadr.csv",index=True)
sample_90.to_csv(f"./datasets/{seed}/euadr/90_sample_val_euadr.csv",index=True)
sample_100.to_csv(f"./datasets/{seed}/euadr/100_sample_val_euadr.csv",index=True)

In [19]:
euadr_unmasked[euadr_unmasked.ENTITY1_TEXT == "fetuin-A"]# and euadr_unmasked.ENTITY1_TEXT == "LRP5"]

,SENTENCE,SENTENCE_MASKED,ASSOCIATION_TYPE,ENTITY1_TEXT,ENTITY2_TEXT,label
350,Male fetuin-A SerSer carriers demonstrate part...,Male @GENE$ SerSer carriers demonstrate partic...,PA,fetuin-A,aortic stiffness,True
351,The fetuin-A polymorphism (Thr256Ser) is assoc...,The @GENE$ polymorphism (Thr256Ser) is associa...,PA,fetuin-A,vascular calcification,True


In [20]:
euadr_unmasked.ENTITY1_TEXT.value_counts()

ME2                       7
IL23R                     7
NOS1AP rs7538490          6
NOD2                      6
PKLR rs3020781            5
                         ..
MBL promoter haplotype    1
epsilon4                  1
epsilon2                  1
Apolipoprotein E          1
EGF                       1
Name: ENTITY1_TEXT, Length: 221, dtype: int64

In [22]:
euadr_unmasked.ENTITY2_TEXT.value_counts()

CD                                    35
papillary thyroid cancer              19
schizophrenia                         17
Crohn's disease                       16
breast cancer                         11
                                      ..
anosmia                                1
Leber congenital amaurosis             1
atopic                                 1
inflammatory and allergic diseases     1
pancreatic cancer                      1
Name: ENTITY2_TEXT, Length: 118, dtype: int64

In [13]:
# unmasked-> entities -> mask -> llama2/gemma -> output
euadr_unmasked.groupby(['ASSOCIATION_TYPE', 'label']).size().unstack(fill_value=0)


label,False,True
ASSOCIATION_TYPE,,
FA,56,37
PA,25,188
SA,7,23


In [52]:
# test_df = pd.DataFrame({
#     'actual_output': ['True', 'False', 'TRUE', 'FALSE', 'true', 'false'],
#     'predicted_output': ['True', 'False', 'TRUE', 'FALSE', 'true', 'false']
# })

# # Normalize case to lower and then convert to integer
# test_df['actual'] = (test_df['actual_output'].str.lower() == 'true').astype(int)
# test_df['predicted'] = (test_df['predicted_output'].str.replace(r"[^TrueFalse]", "", regex=True).str.lower() == 'true').astype(int)


In [53]:
# test_df

### llama2 - EUADR

In [326]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score


In [327]:
llama2_gad_out = pd.read_csv("/home/016651544/llama2_euadr/test_df_10_25_FT_model_euadr.csv")
llama2_gad_out.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,True


In [328]:
def change_nan(row):
    if pd.isna(row[2]) :# or row[2].lower() == "nan":
        return not row[1]
    elif "true" in row[2].lower():
        return True
    elif "false" in row[2].lower():
        return False
    else:
        return not row[1]

In [329]:
print(llama2_gad_out.isna().sum())
llama2_gad_out.dropna(inplace=True)


sentence            0
actual_output       0
predicted_output    0
dtype: int64


In [330]:
llama2_gad_out.shape

(355, 3)

In [331]:
llama2_gad_out.predicted_output.value_counts()

predicted_output
True     239
False    116
Name: count, dtype: int64

In [332]:
# llama2_gad_out.to_csv("/home/016651544/model_predictions_files/llama2_7b_euadr_predictions.csv",index=False)


In [333]:
# Generate classification report
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
actual = llama2_gad_out['actual_output'].astype(str)
predicted = llama2_gad_out['predicted_output'].astype(str)
# Calculate Precision, Recall, and F1 score
precision = precision_score(actual, predicted, average='macro')
recall = recall_score(actual, predicted, average='macro')
f1 = f1_score(actual, predicted, average='macro')
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 score: ",f1)
report = classification_report(actual, predicted)
print(report)

Precision:  0.894459673928726
Recall:  0.9488221291964212
F1 score:  0.9152412877594095
              precision    recall  f1-score   support

       False       0.79      0.99      0.88        93
        True       1.00      0.91      0.95       262

    accuracy                           0.93       355
   macro avg       0.89      0.95      0.92       355
weighted avg       0.94      0.93      0.93       355



### GEMMA 2b

In [334]:
import pandas as pd
gemma_gad_out = pd.read_csv("/home/016651544/gemma/test_df_10000_50_FT_model_gemma_euadr.csv")
gemma_gad_out.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,True


In [335]:
# pd.read_csv("/home/016651544/gemma/euadr_unmasked.csv").SENTENCE[0]

In [336]:
# gemma_gad_out.sentence[0],gemma_gad_out.actual_output[1] 

In [337]:
# Apply the function to each row in the DataFrame
gemma_gad_out.predicted_output = gemma_gad_out.apply(lambda x: change_nan(x),axis=1)

In [338]:
print(gemma_gad_out.isna().sum())
# gemma_gad_out.dropna(inplace=True)


sentence            0
actual_output       0
predicted_output    0
dtype: int64


In [339]:
gemma_gad_out

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,True
...,...,...,...
350,Male @GENE$ SerSer carriers demonstrate partic...,True,True
351,The @GENE$ polymorphism (Thr256Ser) is associa...,True,True
352,@GENE$ gene expression in @DISEASE$ cells sign...,True,True
353,Role of @GENE$ gene in the development of panc...,True,True


In [340]:
gemma_gad_out.shape

(355, 3)

In [341]:
gemma_gad_out.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,True


In [342]:
print(gemma_gad_out.isna().sum())
gemma_gad_out.dropna(inplace=True)


sentence            0
actual_output       0
predicted_output    0
dtype: int64


In [343]:
gemma_gad_out.predicted_output.value_counts(),gemma_gad_out.actual_output.value_counts(),


(predicted_output
 True     268
 False     87
 Name: count, dtype: int64,
 actual_output
 True     262
 False     93
 Name: count, dtype: int64)

In [344]:
# gemma_gad_out.to_csv("/home/016651544/model_predictions_files/gemma_2b_euadr_predictions.csv",index=False)


In [319]:
# Generate classification report
from sklearn.metrics import precision_score, recall_score, f1_score
actual = gemma_gad_out['actual_output']
predicted = gemma_gad_out['predicted_output']
# Calculate Precision, Recall, and F1 score
precision = precision_score(actual, predicted, average='macro')
recall = recall_score(actual, predicted, average='macro')
f1 = f1_score(actual, predicted, average='macro')
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 score: ",f1)
report = classification_report(gemma_gad_out['actual_output'], gemma_gad_out['predicted_output'])
print(report)

Precision:  0.5472636815920398
Recall:  0.5452269555938603
F1 score:  0.5460167714884696
              precision    recall  f1-score   support

       False       0.33      0.31      0.32        93
        True       0.76      0.78      0.77       262

    accuracy                           0.66       355
   macro avg       0.55      0.55      0.55       355
weighted avg       0.65      0.66      0.65       355



### GEMMA 7b - EUADR

In [345]:
import pandas as pd
gemma_gad_out = pd.read_csv("/home/016651544/gemma/euadr/test_df_10000_25_FT_model_euadr_1_1_7b_gemma.csv")
gemma_gad_out.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,NaN


In [346]:
# Apply the function to each row in the DataFrame
gemma_gad_out.predicted_output = gemma_gad_out.apply(lambda x: change_nan(x),axis=1)


In [347]:
print(gemma_gad_out.isna().sum())
# gemma_gad_out.dropna(inplace=True)


sentence            0
actual_output       0
predicted_output    0
dtype: int64


In [348]:
gemma_gad_out

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,False
...,...,...,...
350,Male @GENE$ SerSer carriers demonstrate partic...,True,True
351,The @GENE$ polymorphism (Thr256Ser) is associa...,True,False
352,@GENE$ gene expression in @DISEASE$ cells sign...,True,False
353,Role of @GENE$ gene in the development of panc...,True,False


In [349]:
gemma_gad_out.shape

(355, 3)

In [350]:
gemma_gad_out.head()

,sentence,actual_output,predicted_output
0,Our work supported @GENE$ genetic variants as ...,True,True
1,"Especially, the SNP @GENE$ and its strongly as...",True,True
2,"Especially, the SNP rs491347 and its strongly ...",True,True
3,The HBS1L-MYB intergenic region on chromosome ...,False,False
4,Fetal haemoglobin (@GENE$) level modifies the ...,True,False


In [351]:
print(gemma_gad_out.isna().sum())
gemma_gad_out.dropna(inplace=True)


sentence            0
actual_output       0
predicted_output    0
dtype: int64


In [352]:
gemma_gad_out.predicted_output.value_counts(),gemma_gad_out.actual_output.value_counts(),


(predicted_output
 True     198
 False    157
 Name: count, dtype: int64,
 actual_output
 True     262
 False     93
 Name: count, dtype: int64)

In [353]:
# gemma_gad_out = gemma_gad_out[gemma_gad_out.predicted_output != "NAN"]#.to_csv("/home/016651544/gemma/euadr_gemma_7b_output.csv",index=False)
# gemma_gad_out.to_csv("/home/016651544/model_predictions_files/gemma_7b_euadr_predictions.csv",index=False)

In [284]:
# Generate classification report
from sklearn.metrics import precision_score, recall_score, f1_score
actual = gemma_gad_out['actual_output']
predicted = gemma_gad_out['predicted_output']
# Calculate Precision, Recall, and F1 score
precision = precision_score(actual, predicted, average='macro')
recall = recall_score(actual, predicted, average='macro')
f1 = f1_score(actual, predicted, average='macro')
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 score: ",f1)
report = classification_report(gemma_gad_out['actual_output'], gemma_gad_out['predicted_output'])
print(report)

Precision:  0.6077494692144374
Recall:  0.6374661413444964
F1 score:  0.5986956521739131
              precision    recall  f1-score   support

       False       0.38      0.65      0.48        93
        True       0.83      0.63      0.72       262

    accuracy                           0.63       355
   macro avg       0.61      0.64      0.60       355
weighted avg       0.72      0.63      0.66       355

